In [2]:
import pandas as pd

In [3]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import re
import functools
from geopy.distance import geodesic
from sklearn.metrics import roc_auc_score
from sklearn.metrics import f1_score
import numpy as np

In [4]:
trfk = pd.read_csv('traffic.csv',parse_dates=['datetime'])
data = pd.read_csv('train.csv', parse_dates=['datetime'])
km = pd.DataFrame(np.arange(1,1501),columns=['road_km'])

In [5]:
def hour_rounder(t):
    return (t.replace(second=0, microsecond=0, minute=0, hour=t.hour)
               +timedelta(hours=t.minute//30))

time_start = '2019-01-01'
time_end = '2020-01-01'
trf = trfk[(trfk.datetime>time_start)&(trfk.datetime<time_end)]
trf = trf[trf.road_id==9]
trf['datetime'] = trf['datetime'].map(lambda x: hour_rounder(x))
trf.reset_index(drop=True, inplace=True)



data['datetime'].map(lambda x: hour_rounder(x))
dat = data[(data.datetime>time_start)&(data.datetime<time_end)]
dat = dat[dat.road_id==9]
dat['datetime'] = dat['datetime'].map(lambda x: hour_rounder(x))
dat.reset_index(drop=True, inplace=True)

def to_time(t):
    if t.hour>4 and t.hour<10:
        td = 0 # утро
    elif t.hour>10 and t.hour < 16:
        td = 1 #день
    elif t.hour >16 and t.hour < 22:
        td = 2 #вечер
    else:
        td = 3 #ночь
    
    return {'m':t.month, 'd':t.day, 'h':t.hour, 'wd':t.weekday(), 'td':td}

time = dat.from_records(dat.datetime.apply(lambda x: to_time(x),).values)
dat = dat.join(time)
dat.head()

trf_time = trf.from_records(trf.datetime.apply(lambda x: to_time(x),).values)
trf = trf.join(trf_time)
trf.head()


,datetime,road_id,road_km,name,data_id,station_id,lane,lane_count,direction,volume,occupancy,speed,latitude,longitude,m,d,h,wd,td
0,2019-01-01 01:00:00,9,527,км 526+700,27457348,41108,1,1,forward,12.0,0.20,38.0,59.638016,40.561815,1,1,1,1,3
1,2019-01-01 01:00:00,9,1219,км 1219+000,27457209,41101,2,3,backward,50.0,1.66,88.3,64.456010,40.603788,1,1,1,1,3
2,2019-01-01 01:00:00,9,679,км 679+000,27741671,41105,1,1,backward,12.0,0.60,86.0,60.733126,41.898933,1,1,1,1,3
3,2019-01-01 01:00:00,9,1151,км 1151+130,27457256,41103,2,2,backward,0.0,0.00,0.0,64.145549,41.611204,1,1,1,1,3
4,2019-01-01 01:00:00,9,1151,км 1151+130,27457256,41103,1,2,forward,6.0,0.23,53.7,64.145549,41.611204,1,1,1,1,3


#Фичи из времени

In [18]:
df = dat
df['i']=1

In [19]:
mean_dtp_by_month = df.groupby(by=['d','m']).i.sum().reset_index().groupby(by='m').mean().i
mean_dtp_by_month = ((mean_dtp_by_month-mean_dtp_by_month.min())/mean_dtp_by_month.max()).reset_index().i.to_dict()

mean_dtp_by_day = df.groupby(by=['d','m']).i.sum().reset_index().groupby(by='d').mean().i
mean_dtp_by_day = ((mean_dtp_by_day-mean_dtp_by_day.min())/mean_dtp_by_day.max()).reset_index().i.to_dict()

mean_dtp_by_weekday = df.groupby(by=['wd','m']).i.sum().reset_index().groupby(by='wd').mean().i
mean_dtp_by_weekday = ((mean_dtp_by_weekday-mean_dtp_by_weekday.min())/mean_dtp_by_weekday.max()).reset_index().i.to_dict()

mean_dtp_by_daytime = df.groupby(by=['td','m']).i.sum().reset_index().groupby(by='td').mean().i
mean_dtp_by_daytime = ((mean_dtp_by_daytime-mean_dtp_by_daytime.min())/mean_dtp_by_daytime.max()).reset_index().i.to_dict()

#Фичи из расстояния

In [107]:
df_km = pd.merge(dat,km, how='outer',on=['road_km'])

df_km.fillna(0,inplace=True)
df_km = df_km[['road_km','man_injured_num','man_dead_num','car_damaged_num','target','m','d','h','i']]

In [108]:
mean_dtp_by_km_1 = df_km.groupby(by='road_km').sum().reset_index().i
mean_dtp_by_km_1  = (mean_dtp_by_km_1/mean_dtp_by_km_1.max()).reset_index()

mean_dtp_by_km_10 = mean_dtp_by_km_1.rolling(10).mean()[10::10].i
mean_dtp_by_km_10 = (mean_dtp_by_km_10/mean_dtp_by_km_10.max()).reset_index()

mean_dtp_by_km_20 = mean_dtp_by_km_1.rolling(20).mean()[20::20].i
mean_dtp_by_km_20 = (mean_dtp_by_km_20/mean_dtp_by_km_20.max()).reset_index()

In [109]:
mdk1 = mean_dtp_by_km_1.i.to_dict()
mdk10 = mean_dtp_by_km_10.i.to_dict()
mdk20 = mean_dtp_by_km_20.i.to_dict()


#ФИЧИ ИЗ ТРАФИКА!

In [110]:
trf.dropna()

,datetime,road_id,road_km,name,data_id,station_id,lane,lane_count,direction,volume,occupancy,speed,latitude,longitude,m,d,h,wd,td
0,2019-01-01 01:00:00,9,527,км 526+700,27457348,41108,1,1,forward,12.0,0.200000,38.00000,59.638016,40.561815,1,1,1,1,3
1,2019-01-01 01:00:00,9,1219,км 1219+000,27457209,41101,2,3,backward,50.0,1.660000,88.30000,64.456010,40.603788,1,1,1,1,3
2,2019-01-01 01:00:00,9,679,км 679+000,27741671,41105,1,1,backward,12.0,0.600000,86.00000,60.733126,41.898933,1,1,1,1,3
3,2019-01-01 01:00:00,9,1151,км 1151+130,27457256,41103,2,2,backward,0.0,0.000000,0.00000,64.145549,41.611204,1,1,1,1,3
4,2019-01-01 01:00:00,9,1151,км 1151+130,27457256,41103,1,2,forward,6.0,0.230000,53.70000,64.145549,41.611204,1,1,1,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590674,2020-01-01 00:00:00,9,231,км 231+400,34600124,40722,2,2,forward,10.0,0.222656,95.61313,57.379030,39.609239,1,1,0,2,3
590675,2020-01-01 00:00:00,9,57,км 57+200,34600019,53225,3,3,forward,119.0,1.531250,98.67500,56.186856,38.035917,1,1,0,2,3
590676,2020-01-01 00:00:00,9,231,км 231+400,34600124,40722,1,2,forward,26.0,0.613281,88.32375,57.379030,39.609239,1,1,0,2,3
590677,2020-01-01 00:00:00,9,231,км 231+400,34600124,40722,2,2,backward,20.0,0.429688,100.57687,57.379030,39.609239,1,1,0,2,3


In [111]:
mean_volume_by_month = trf.groupby(by=['m']).volume.mean().reset_index().volume.to_dict()
mean_speed_by_month = trf.groupby(by=['m']).speed.mean().reset_index().speed.to_dict

In [112]:
t = trf.groupby(by=['road_km']).volume.mean().reset_index()
volume = (pd.merge(km,t,how='outer').interpolate(method='linear').volume*2).iloc[:1500]
volume.fillna(500,inplace=True)

In [113]:
def get_param(param,h):
    hvol = trf.groupby(by=['road_km',h])[param].mean().reset_index()
    hour_volume = pd.DataFrame()
    for p in trf[h].unique():
        v = hvol[hvol[h]==p]
        hour_volume[p] = (pd.merge(km,v,how='outer').interpolate(method='linear')[param]).iloc[:1500]
    hour_volume= hour_volume.fillna(hour_volume.median())
    return hour_volume.to_dict()

In [114]:
speed_h = get_param('speed','h')

In [115]:
trf.groupby(by=['road_km','h']).volume.mean().reset_index()
mean_dtp_by_weekday = df.groupby(by=['wd','m']).i.sum().reset_index().groupby(by='wd').mean().i
mean_dtp_by_weekday = ((mean_dtp_by_weekday-mean_dtp_by_weekday.min())/mean_dtp_by_weekday.max()).reset_index().i.to_dict()

In [116]:
def prepare_data(x):
    kmdtp1 = mdk1.get(x.road_km) #koef km dtp
    kmdtp10 = mdk10.get(round(x.road_km/20)) #koef 10 km dtp
    kmdtp20 = mdk20.get(round(x.road_km/40)) #koef 20 km dtp
    if kmdtp1 == None:
        kmdtp1 = 0
    if kmdtp10 == None:
        kmdtp10 = 0
    if kmdtp20 == None:
        kmdtp20 = 0
    
    mdtp = mean_dtp_by_month.get(x.m-1) #koef dtp at day in month
    ddtp = mean_dtp_by_day.get(x.d-1) 
    wdtp = mean_dtp_by_weekday.get(x.wd) 
    dtdtp = mean_dtp_by_daytime.get(x.td)
    
    
    return {'mdtp1':kmdtp1, 'mdtp10':kmdtp10, 'mdtp20':kmdtp20, 'monthdtp':mdtp, 'ddtp':ddtp, 'wdtp':wdtp, 'dtdtp':dtdtp}




nd = dat[['m','d','h','road_km','wd','td']].apply(lambda x:prepare_data(x),axis=1)
ndi = pd.DataFrame.from_records(nd)

In [117]:
dat3 = dat.reset_index(drop=True)
trf3 = trf.reset_index(drop=True)

In [118]:
nd3 = dat3[['m','d','h','road_km','wd','td']].apply(lambda x:prepare_data(x),axis=1)
nd3 = pd.DataFrame.from_records(nd3)
nd3

,mdtp1,mdtp10,mdtp20,monthdtp,ddtp,wdtp,dtdtp
0,0.053435,0.081744,0.0664,0.038803,0.094340,0.075334,0.353877
1,1.000000,1.000000,1.0000,0.000000,0.146226,0.024301,0.353877
2,0.343511,0.564033,1.0000,0.138532,0.089623,0.036452,0.353877
3,0.106870,1.000000,1.0000,0.205818,0.193396,0.211422,0.353877
4,0.282443,0.564033,1.0000,0.065242,0.122642,0.030377,0.335321
...,...,...,...,...,...,...,...
4917,0.076336,0.040872,0.0488,0.146448,0.207547,0.036452,0.353877
4918,0.122137,0.702997,0.2544,0.093014,0.245283,0.036452,0.475812
4919,0.099237,0.004087,0.0128,0.224751,0.141509,0.211422,0.353877
4920,1.000000,1.000000,1.0000,0.331090,0.221698,0.030377,0.335321


In [121]:
trf3 = trf3.groupby(by='data_id').mean().reset_index(drop=True)

speed_median_month = trf3.groupby(by=['m']).speed.median().to_dict()
speed_median_wd = trf3.groupby(by=['wd']).speed.median().to_dict()
speed_median_h = trf3.groupby(by=['h']).speed.median().to_dict()
speed_median_td = trf3.groupby(by=['td']).speed.median().to_dict()

volume_median_month = trf3.groupby(by=['m']).volume.median().to_dict()
volume_median_wd = trf3.groupby(by=['wd']).volume.median().to_dict()
volume_median_h = trf3.groupby(by=['h']).volume.median().to_dict()
volume_median_td = trf3.groupby(by=['td']).volume.median().to_dict()

occupancy_median_month = trf3.groupby(by=['m']).occupancy.median().to_dict()
occupancy_median_wd = trf3.groupby(by=['wd']).occupancy.median().to_dict()
occupancy_median_h = trf3.groupby(by=['h']).occupancy.median().to_dict()
occupancy_median_td = trf3.groupby(by=['td']).occupancy.median().to_dict()

In [122]:
d3 = pd.concat([dat3.reset_index(drop=True), nd3],axis=1)

In [124]:
mergeddata = pd.merge(trf3,d3,how='outer',on=['road_km','m','d','h','wd','td','road_id'])
mdata = mergeddata[['road_id','road_km','lane_count','volume','occupancy','speed','m','d','h','wd','td','target','mdtp1',
       'mdtp10', 'mdtp20', 'monthdtp', 'ddtp', 'wdtp', 'dtdtp']]
mdata.head()

,road_id,road_km,lane_count,volume,occupancy,speed,m,d,h,wd,td,target,mdtp1,mdtp10,mdtp20,monthdtp,ddtp,wdtp,dtdtp
0,9.0,254.0,3.0,32.333333,0.568359,82.413333,1.0,1.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9.0,342.0,2.0,8.000000,0.187500,37.112500,1.0,1.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,126.0,1.0,43.500000,1.099609,74.977500,1.0,1.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9.0,342.0,2.0,3.000000,0.166016,43.457500,1.0,1.0,2.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9.0,181.0,1.0,33.500000,1.000000,62.915000,1.0,1.0,1.0,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [126]:
def prepare_full_data(x):
    kmdtp1 = mdk1.get(x.road_km) #koef km dtp
    kmdtp10 = mdk10.get(round(x.road_km/20)) #koef 10 km dtp
    kmdtp20 = mdk20.get(round(x.road_km/40)) #koef 20 km dtp
    if kmdtp1 == None:
        kmdtp1 = 0
    if kmdtp10 == None:
        kmdtp10 = 0
    if kmdtp20 == None:
        kmdtp20 = 0
    
    mdtp = mean_dtp_by_month.get(x.m-1) #koef dtp at day in month
    ddtp = mean_dtp_by_day.get(x.d-1) 
    wdtp = mean_dtp_by_weekday.get(x.wd) 
    dtdtp = mean_dtp_by_daytime.get(x.td)
    
    lane_count = 2 if x.lane_count else x.lane_count
    volume = volume_median_td.get(x.td) 
    speed = speed_median_td.get(x.td) 
    occupancy = occupancy_median_td.get(x.td)
        
        
    
    
    
    return {'road_id':x.road_id, 'road_km':x.road_km, 'lane_count':lane_count, 'volume':volume, 'occupancy':occupancy, 'speed':speed,
            'm':x.m, 'd':x.d, 'h':x.h, 'wd':x.wd, 'td':x.td,
            'mdtp1':kmdtp1, 'mdtp10':kmdtp10, 'mdtp20':kmdtp20, 'monthdtp':mdtp, 'ddtp':ddtp, 'wdtp':wdtp, 'dtdtp':dtdtp, 'target':x.target}


In [127]:
md = pd.DataFrame.from_records(mdata.apply(lambda x: prepare_full_data(x),axis=1).values)
md.fillna(0,inplace=True)

In [130]:
tmp.target.value_counts()
tmp_train, tmp_test = train_test_split(tmp, test_size=0.3,random_state=24)
print("Размер выборки для обучения: ", tmp_train.shape)
print("Размер выборки для тестирования: ", tmp_test.shape)

Размер выборки для обучения:  (13820, 19)
Размер выборки для тестирования:  (5924, 19)


In [132]:
from sklearn import datasets, ensemble

In [133]:
from sklearn.metrics import f1_score
from sklearn import svm



tmp_train_x = tmp_train.drop('target',axis=1)
tmp_train_y = tmp_train['target']

tmp_test_x = tmp_test.drop('target',axis=1)
tmp_test_y = tmp_test['target']

In [138]:
xgb = ensemble.GradientBoostingClassifier(n_estimators=100,max_depth=3)
predict_target = xgb.fit(tmp_train_x, tmp_train_y)
# оценка модели на реальных данных тестовой выборки 
f1_score(tmp_test_y, xgb.predict(tmp_test_x), average='macro')

0.743226660373769

In [139]:
from sklearn import datasets, ensemble

In [36]:
test = pd.read_csv('test.csv',parse_dates=['datetime'])
traffic = pd.read_csv('traffic.csv',parse_dates=['datetime'])

In [37]:
test['datetime'] = test['datetime'].map(lambda x: hour_rounder(x))
test.reset_index(drop=True, inplace=True)
test_time = test.from_records(test.datetime.apply(lambda x: to_time(x),).values)
test = test.join(test_time)
test.head()

,datetime,road_id,road_km,target,m,d,h,wd,td
0,2020-01-01 01:00:00,9,0,NaN,1,1,1,2,3
1,2020-01-01 01:00:00,9,1,NaN,1,1,1,2,3
2,2020-01-01 01:00:00,9,2,NaN,1,1,1,2,3
3,2020-01-01 01:00:00,9,3,NaN,1,1,1,2,3
4,2020-01-01 01:00:00,9,4,NaN,1,1,1,2,3


In [38]:
test.to_pickle('test_time.pickle')

In [39]:
test['lane_count']=np.NaN
test['volume']=np.NaN
test['speed']=np.NaN
test['occupancy']=np.NaN

In [41]:
test

,datetime,road_id,road_km,target,m,d,h,wd,td,lane_count,volume,speed,occupancy
0,2020-01-01 01:00:00,9,0,NaN,1,1,1,2,3,NaN,NaN,NaN,NaN
1,2020-01-01 01:00:00,9,1,NaN,1,1,1,2,3,NaN,NaN,NaN,NaN
2,2020-01-01 01:00:00,9,2,NaN,1,1,1,2,3,NaN,NaN,NaN,NaN
3,2020-01-01 01:00:00,9,3,NaN,1,1,1,2,3,NaN,NaN,NaN,NaN
4,2020-01-01 01:00:00,9,4,NaN,1,1,1,2,3,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1729446,2020-10-25 23:00:00,14,1545,NaN,10,25,23,6,3,NaN,NaN,NaN,NaN
1729447,2020-10-25 23:00:00,14,1546,NaN,10,25,23,6,3,NaN,NaN,NaN,NaN
1729448,2020-10-25 23:00:00,14,1547,NaN,10,25,23,6,3,NaN,NaN,NaN,NaN
1729449,2020-10-25 23:00:00,14,1548,NaN,10,25,23,6,3,NaN,NaN,NaN,NaN


In [42]:
prepare_test = pd.DataFrame.from_records(test.apply(lambda x: prepare_full_data(x),axis=1).values)

In [43]:
prepare_test.to_pickle('prepare_test.pickle')

In [45]:
prepare_test.head(20)

,road_id,road_km,lane_count,volume,occupancy,speed,m,d,h,wd,td,mdtp1,mdtp10,mdtp20,monthdtp,ddtp,wdtp,dtdtp,target
0,9,0,2,90.0,3.7825,73.95,1,1,1,2,3,0.061069,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
1,9,1,2,90.0,3.7825,73.95,1,1,1,2,3,0.038168,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
2,9,2,2,90.0,3.7825,73.95,1,1,1,2,3,0.053435,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
3,9,3,2,90.0,3.7825,73.95,1,1,1,2,3,0.022901,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
4,9,4,2,90.0,3.7825,73.95,1,1,1,2,3,0.030534,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
5,9,5,2,90.0,3.7825,73.95,1,1,1,2,3,0.030534,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
6,9,6,2,90.0,3.7825,73.95,1,1,1,2,3,0.015267,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
7,9,7,2,90.0,3.7825,73.95,1,1,1,2,3,0.015267,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
8,9,8,2,90.0,3.7825,73.95,1,1,1,2,3,0.015267,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN
9,9,9,2,90.0,3.7825,73.95,1,1,1,2,3,0.053435,0.051771,0.3616,0.227588,0.174528,0.082625,0.475812,NaN


In [250]:
pr = xgb.predict(prepare_test.drop('target',axis=1))

In [251]:
ans = test
ans.target = pr
ans[['datetime','road_id','road_km','target']].to_csv('xgb4.csv',index=False)

/Users/vaceslavnazvanov/Emergency/lib/python3.8/site-packages/pandas/core/generic.py:5494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
